# AxCoRSMRI sample notebook

Self-Supervised Realistic Through-Plane MRI Super Resolution from Clinical 2D Axial and Coronal Acquisition.

[For more information, please see the GitHub repository.](https://github.com/TechnionComputationalMRILab/AxCorSRMRI)

In [1]:
## Imports
import torch
assert torch.cuda.is_available()

import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

## Data Preprocess

For first time users, please prepare your data accordingly. 

Create a where each case ID has two files - one for axial and one for coronal. The files should be in nifti format (.nii.gz). The files should be named as follows: "case_id_AXIAL.nii.gz" and "case_id_CORONAL.nii.gz".

For example:
```
    - data_folder
        - 1
            - 1_AXIAL.nii.gz
            - 1_CORONAL.nii.gz
        - 2
            - 2_AXIAL.nii.gz
            - 2_CORONAL.nii.gz
        - 3
            - 3_AXIAL.nii.gz
            - 3_CORONAL.nii.gz
        - 4
            - 4_AXIAL.nii.gz
            - 4_CORONAL.nii.gz
        - 5
            - 5_AXIAL.nii.gz
            - 5_CORONAL.nii.gz
```

Note that you need to have write permissions in the `data_folder` directory, as the code will write the preprocessed data in the same directory. The paths require a trailing slash.

In [2]:
# %%time
# # Resample isotropic NIFTI coronal files
# from axcorsrmri import resample_cases

# path_to_data_files = r"./data/"
# resample_cases(path_dir = path_to_data_files)

In [3]:
# %%time

# # Create DB file with isotropic, coronal and axial files paths
# from axcorsrmri import create_database

# path_to_data_files = r"./data/"
# create_database(
#     path_to_data_files,
#     train_frac=0.8,
#     test_frac=0.1,
#     num_folds=1
# )

## Param Initializaion

Here you can define the main parameters for the model training. Please refer to `Parameters_dictionary.txt` for more details of each parameter.

In [4]:
from axcorsrmri import parser_setup

override_args = {
    "path_to_set": r"./data/",
    "path_to_results": r"./results/",
    "amount_of_files":20,
    "batch_size":12,
    "loss":"L2",
    "gpu_device":"0",
    "amount_of_slices":3,
    "title":"Test",
    "total_samples":100,
    "patch_size":48,
    "epochs": 20,
    "lr_g":0.0001,
    "lr_d": 0.0001,
    "scheduler": "const",
    "max_workers_train": 12,
    "max_workers_valid": 30,
    "valid_batch_size": 40,
    "adversarial_weight_I": 0.02,
    "adversarial_weight_E": 0.02,
    "d_optimizer_step_size":40 ,
    "g_optimizer_step_size": 160,
    "val_epoch": 5,
    "image_save_freq_batch": 100,
    "mage_save_freq_epoch": 5,
    "save_tensor":True,
    "save_nifti":True
}

args = parser_setup(override_args)

## Dataset Creation

Initialize the training, validation and test datasets for the model training. In addition, the function creates a new folder in `path_to_results` to save the results and logs for tracking with tensorboard.


In [5]:
%%time
from axcorsrmri import initialize_data
dl_train, dl_valid_lr, dl_valid_hr, dl_test_lr, dl_test_hr, result_dir, writer, config = initialize_data(args)

Building and creating ESRT model from scratch.
Loading the training and validation datasets...
Loading the training and validation datasets...
Loaded training and validation datasets successfully.
Finished data preparation and parameters initialization.
CPU times: user 2min 52s, sys: 1min 32s, total: 4min 24s
Wall time: 1min 59s


## Training, validation, and testing
Initalize the model and model parameters, and then train it on `dl_train` and validate it on `dl_valid_lr` and `dl_valid_hr`. After the training is done, it is tested on `dl_test_lr` and `dl_test_hr`.

In [6]:
%%time

from axcorsrmri import training_validation_test
training_validation_test(dl_train, dl_valid_lr, dl_valid_hr, dl_test_lr, dl_test_hr, result_dir, writer, config)

Building model...
Built model successfully.
Defining all optimizer functions...
Defined all optimizer functions successfully.
Defining all optimizer scheduler functions...
Defined all optimizer scheduler functions successfully.
Defining all loss functions...
Defined all loss functions successfully.


  0%|          | 0/20 [00:00<?, ?it/s]

Starting model training...


  0%|          | 0/20 [00:05<?, ?it/s]


NotImplementedError: Could not run 'aten::_amp_foreach_non_finite_check_and_unscale_' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_amp_foreach_non_finite_check_and_unscale_' is only available for these backends: [CUDA, BackendSelect, Named, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, UNKNOWN_TENSOR_TYPE_ID, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, Autocast, Batched, VmapMode].

CUDA: registered at /pytorch/build/aten/src/ATen/RegisterCUDA.cpp:20674 [kernel]
BackendSelect: fallthrough registered at /pytorch/aten/src/ATen/core/BackendSelectFallbackKernel.cpp:3 [backend fallback]
Named: registered at /pytorch/aten/sr

## Reconstruct SR Volumes
Apply the SR model on all `isotropic` files in the model.


In [7]:
from axcorsrmri import test_parser_setup

override_args_test = {
    "path_to_set": r"./data/",
    "path_to_results": r"./results/",
    "path_to_trained_model": r"./Test_06_06_2024_11_47/",
    "amount_of_files":20,
    "batch_size":12,
    "gpu_device":"0,1",
    "amount_of_slices":3,
    "title":"Test",
    "save_tensor":True,
    "save_nifti":True,
}

test_args = test_parser_setup(override_args_test)

In [8]:
from axcorsrmri import reconstruct_SR_volumes_in_folder

reconstruct_SR_volumes_in_folder(test_args)

Creating the dataset...


KeyboardInterrupt: 